In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/My Drive/emotions_intensity'

/content/drive/My Drive/emotions_intensity


In [ ]:
!pip install transformers

In [ ]:
import sys
import re
import html
import nltk
from nltk import word_tokenize
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
  

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

FLAGS = re.MULTILINE | re.DOTALL

# Lemmatize with POS Tag
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def hashtag(text):
    text = text.group()
    hashtag_body = text[1:]
    if hashtag_body.isupper():
        result = "<hashtag> {} <allcaps>".format(hashtag_body)
    else:
        result = " ".join(["<hashtag>"] + re.split(r"(?=[A-Z])", hashtag_body, flags=FLAGS))
    return result

def allcaps(text):
    text = text.group()
    return text.lower() + " <allcaps> "

def remove_stopwords(string):
    split_string = \
        [word for word in string.split()
         if word not in stopwords.words('english')]
    
    return " ".join(split_string)



def clean_str(string):  
    string = html.unescape(string)
    string = string.replace("\\n", " ")
    string = string.replace("_NEG", "")
    string = string.replace("_NEGFIRST", "")
    string = re.sub(r"@[A-Za-z0-9_s(),!?\'\`]+", "", string) # removing any twitter handle mentions
    #string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"#", "", string)
    string = re.sub(r"\*", "", string)
    string = re.sub(r"\'s", "", string)
    string = re.sub(r"\'m", " am", string)
    string = re.sub(r"\'ve", " have", string)
    string = re.sub(r"n\'t", " not", string)
    string = re.sub(r"\'re", " are", string)
    string = re.sub(r"\'d", " would", string)
    string = re.sub(r"\'ll", " will", string)
    string = re.sub(r",", "", string)
    string = re.sub(r"!", " !", string)
    string = re.sub(r"\(", "", string)
    string = re.sub(r"\)", "", string)
    string = re.sub(r"\?", " ?", string)
    string = re.sub(r"\s{2,}", " ", string)
    # remove numerics
    string = ''.join([i for i in string if not i.isdigit()])
    #return string.strip().lower()
    return remove_stopwords(string.strip().lower())


def preprocess(text):
    # Different regex parts for smiley faces
    eyes = r"[8:=;]"
    nose = r"['`\-]?"
		
    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)	
	
    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", " <url> ")
    text = re_sub(r"/"," / ")
    text = re_sub(r"@\w+", " <user> ")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), " <smile> ")
    text = re_sub(r"{}{}p+".format(eyes, nose), " <lolface> ")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), " <sadface> ")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), " <neutralface> ")
    text = re_sub(r"<3","<heart>")
    #text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", " <number> ")
    # text = re_sub(r"#\S+", hashtag)
    text = re_sub(r"([!?.]){2,}", r"\1 <repeat> ")
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong> ")

    ## -- I just don't understand why the Ruby script adds <allcaps> to everything so I limited the selection.
    # text = re_sub(r"([^a-z0-9()<>'`\-]){2,}", allcaps)
    text = re_sub(r"([A-Z]){2,}", allcaps)
    # lemmatize the text
    lemmatizer = WordNetLemmatizer()
    text = " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)])
    text = clean_str(text)

    return text.lower()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
print(preprocess("How the fu*k! Who the heck! moved my fridge!... should I knock the landlord door. #angry #mad ##"))

fuk ! heck ! move fridge . < repeat > knock landlord door . angry mad


In [ ]:
def read_data(data_file_path):

    data_list = list()
    with open(data_file_path) as input_file:
        for line in input_file:
            line = line.strip()
            array = line.split('\t')
            data_list.append((array[0], preprocess(array[1]), array[2], float(array[3])))
    return data_list

In [ ]:
import random
raw_training_data = read_data("data/joy-ratings-0to1.train.txt")
raw_test_data = read_data("data/joy-ratings-0to1.test.gold.txt")
# shuffle
random.shuffle(raw_training_data)
# print a few samples
print(raw_training_data[:5])

[('30709', '< user > compatibility key lower hedonic volatility .', 'joy', 0.25), ('30700', 'one time saw rachel glee tell someone job pole say th grade boy look confuse', 'joy', 0.271), ('30525', 'always ready .', 'joy', 0.407), ('30347', '< user > trust dear want make anything invisible hang wet paint hilarity ensue .', 'joy', 0.521), ('30375', 'watch amaze live.ly broadcast < user > lively musically', 'joy', 0.51)]


In [ ]:
train_sentences = []
y_train = []
for sample in raw_training_data:
  train_sentences.append(sample[1])
  y_train.append(sample[3])

test_sentences = []
y_test = []
for sample in raw_test_data:
  test_sentences.append(sample[1])
  y_test.append(sample[3])

In [ ]:
#define scorer
def pearsonr(y_true, y_pred):
    """Scoring function: Calculates a Pearson correlation coefficient.

    Parameters
    ----------
    y_true: numpy.ndarray {n_samples}
        True class labels
    y_pred: numpy.ndarray {n_samples}
        Predicted class labels by the estimator
    Returns
    -------
    fitness: float
        Returns a float value indicating Pearson correlation coefficient
    """
    from scipy.stats import pearsonr
    r = pearsonr(y_true, y_pred)[0]
    if np.isnan(r):
        r = 0.0
    return r

from sklearn.metrics import make_scorer
pear_score = make_scorer(pearsonr)

## With Bag of Words Embeddings

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
 
 
 
CountVec = CountVectorizer(ngram_range=(1,1), # to use bigrams ngram_range=(2,2)
                           stop_words='english')
#transform
Count_data = CountVec.fit_transform(train_sentences)
 
#create dataframe
cv_train_dataframe=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())
cv_train_dataframe.head()

,__,_full_plus_by_gora,_ø,ab,abby,ability,abla,able,absolute,absolutely,abuse,abysmal,acapella,accept,accepts,account,accurate,achieve,acquire,act,actively,actorslife,actual,actually,adcomm,add,addict,addiction,admits,adorable,adore,adrenaline,adult,adventure,advert,advocate,aesthetically,af,afflict,afford,...,yoga,yoosung,young,yourebeautiful,youtube,yr,yrs,yt,yup,yvr,zephaniah,zeth,zoolander,ªðÿ,ºï,ºðÿ,¼ï,¼ðÿ,âœ,âš,âž,âˆ,ï¼,ï¼ˆâ,ðµð½ñœðð,ðÿ,ðÿœ¾ðÿ,ùš,ùšùˆù,ùˆø,œbe,œclick,œdonâ,œpause,œset,œthe,œwhen,œðÿ,šðÿ,žðÿ
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#transform
Count_data = CountVec.transform(test_sentences)
 
#create dataframe
cv_test_dataframe=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())
cv_test_dataframe.head()

,__,_full_plus_by_gora,_ø,ab,abby,ability,abla,able,absolute,absolutely,abuse,abysmal,acapella,accept,accepts,account,accurate,achieve,acquire,act,actively,actorslife,actual,actually,adcomm,add,addict,addiction,admits,adorable,adore,adrenaline,adult,adventure,advert,advocate,aesthetically,af,afflict,afford,...,yoga,yoosung,young,yourebeautiful,youtube,yr,yrs,yt,yup,yvr,zephaniah,zeth,zoolander,ªðÿ,ºï,ºðÿ,¼ï,¼ðÿ,âœ,âš,âž,âˆ,ï¼,ï¼ˆâ,ðµð½ñœðð,ðÿ,ðÿœ¾ðÿ,ùš,ùšùˆù,ùˆø,œbe,œclick,œdonâ,œpause,œset,œthe,œwhen,œðÿ,šðÿ,žðÿ
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


##### SVM Regression

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.svm import SVR
param_grid = {
            'kernel': ['rbf'],
            'C': [0.1, 1, 100, 1000],
            'epsilon': [0.0001, 0.001, 0.01, 0.1, 1],
            'gamma': [0.0001, 0.001, 0.1, 1, 5]
        }
gsc = GridSearchCV(
        estimator=SVR(),
        param_grid=param_grid,
        cv=5, scoring=pear_score, 
        verbose=10000000, n_jobs=-1)

grid_result = gsc.fit(cv_train_dataframe, y_train)

best_params = grid_result.best_params_

print(best_params)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   15.2s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  

In [ ]:
from sklearn.svm import SVR
svr = SVR(**best_params)
svr.fit(cv_train_dataframe,y_train)

SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.01, gamma=0.0001,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr,pearsonr

y_pred = svr.predict(cv_test_dataframe)
print(mean_squared_error(y_pred,y_test))

corr, _ = pearsonr(y_pred, y_test)
s_corr,_ = spearmanr(y_pred,y_test)
print("PEARSON: " + str(corr))

print("SPEARMAN: " + str(s_corr))

0.03521509522617036
PEARSON: 0.5140415571526249
SPEARMAN: 0.5116266396193618


Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeRegressor
param_grid = {"min_samples_split": [10, 20, 40],
              "max_depth": [2, 6, 8],
              "min_samples_leaf": [20, 40, 100],
              "max_leaf_nodes": [5, 20, 100],
              }

gsc = GridSearchCV(
        estimator=DecisionTreeRegressor(),
        param_grid=param_grid,
        cv=5, scoring=pear_score, 
        verbose=10000000, n_jobs=-1)

grid_result = gsc.fit(cv_train_dataframe, y_train)

best_params = grid_result.best_params_

print(best_params)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0999s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1646s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    0.8s
[Parallel(n_jo

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor(**best_params)

dtr.fit(cv_train_dataframe,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=6,
                      max_features=None, max_leaf_nodes=20,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=20, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [ ]:
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr,pearsonr

y_pred = dtr.predict(cv_test_dataframe)
print(mean_squared_error(y_pred,y_test))

corr, _ = pearsonr(y_pred, y_test)
s_corr,_ = spearmanr(y_pred,y_test)
print("PEARSON: " + str(corr))

print("SPEARMAN: " + str(s_corr))

0.04270956141870682
PEARSON: 0.32197798033187647
SPEARMAN: 0.3310150063301089


Multi-Layer Perceptron

In [ ]:
from sklearn.neural_network import MLPRegressor
param_grid = {
    "hidden_layer_sizes": [(i,) for i in range(10,101,30)],
    "activation": [ "tanh", "relu"],
    "solver": ["adam"],
    "learning_rate":["invscaling", "adaptive"],
    "learning_rate_init":[0.005,0.0005,0.00005],
    "max_iter":[200,500,900],
}
print(param_grid)
gsc = GridSearchCV(
        estimator=MLPRegressor(),
        param_grid=param_grid,
        cv=5, scoring=pear_score, 
        verbose=10000000, n_jobs=-1)
grid_result = gsc.fit(cv_train_dataframe, y_train)


{'hidden_layer_sizes': [(10,), (40,), (70,), (100,)], 'activation': ['tanh', 'relu'], 'solver': ['adam'], 'learning_rate': ['invscaling', 'adaptive'], 'learning_rate_init': [0.005, 0.0005, 5e-05], 'max_iter': [200, 500, 900]}
Fitting 5 folds for each of 144 candidates, totalling 720 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.9s
[Parallel(n_job

In [ ]:
best_params = grid_result.best_params_

print(best_params)

{'activation': 'tanh', 'hidden_layer_sizes': (10,), 'learning_rate': 'invscaling', 'learning_rate_init': 0.005, 'max_iter': 200, 'solver': 'adam'}


In [ ]:
mlp = MLPRegressor(**best_params)
mlp.fit(cv_train_dataframe,y_train)

MLPRegressor(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(10,), learning_rate='invscaling',
             learning_rate_init=0.005, max_fun=15000, max_iter=200,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [ ]:
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr,pearsonr

y_pred = mlp.predict(cv_test_dataframe)
print(mean_squared_error(y_pred,y_test))

corr, _ = pearsonr(y_pred, y_test)
s_corr,_ = spearmanr(y_pred,y_test)
print("PEARSON: " + str(corr))

print("SPEARMAN: " + str(s_corr))

0.05730528146643248
PEARSON: 0.3895436003823237
SPEARMAN: 0.36339829491135317


## With TF-IDF Embeddings

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
 
 
 
CountVec = TfidfVectorizer(ngram_range=(1,1), # to use bigrams ngram_range=(2,2)
                           stop_words='english')
#transform
Count_data = CountVec.fit_transform(train_sentences)
 
#create dataframe
cv_train_dataframe=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())
cv_train_dataframe.head()

,__,_full_plus_by_gora,_ø,ab,abby,ability,abla,able,absolute,absolutely,abuse,abysmal,acapella,accept,accepts,account,accurate,achieve,acquire,act,actively,actorslife,actual,actually,adcomm,add,addict,addiction,admits,adorable,adore,adrenaline,adult,adventure,advert,advocate,aesthetically,af,afflict,afford,...,yoga,yoosung,young,yourebeautiful,youtube,yr,yrs,yt,yup,yvr,zephaniah,zeth,zoolander,ªðÿ,ºï,ºðÿ,¼ï,¼ðÿ,âœ,âš,âž,âˆ,ï¼,ï¼ˆâ,ðµð½ñœðð,ðÿ,ðÿœ¾ðÿ,ùš,ùšùˆù,ùˆø,œbe,œclick,œdonâ,œpause,œset,œthe,œwhen,œðÿ,šðÿ,žðÿ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#transform
Count_data = CountVec.transform(test_sentences)

#create dataframe
cv_test_dataframe=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())
cv_test_dataframe.head()

,__,_full_plus_by_gora,_ø,ab,abby,ability,abla,able,absolute,absolutely,abuse,abysmal,acapella,accept,accepts,account,accurate,achieve,acquire,act,actively,actorslife,actual,actually,adcomm,add,addict,addiction,admits,adorable,adore,adrenaline,adult,adventure,advert,advocate,aesthetically,af,afflict,afford,...,yoga,yoosung,young,yourebeautiful,youtube,yr,yrs,yt,yup,yvr,zephaniah,zeth,zoolander,ªðÿ,ºï,ºðÿ,¼ï,¼ðÿ,âœ,âš,âž,âˆ,ï¼,ï¼ˆâ,ðµð½ñœðð,ðÿ,ðÿœ¾ðÿ,ùš,ùšùˆù,ùˆø,œbe,œclick,œdonâ,œpause,œset,œthe,œwhen,œðÿ,šðÿ,žðÿ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.272373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


SVM Regression

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.svm import SVR
param_grid = {
            'kernel': ['rbf'],
            'C': [0.1, 1, 100, 1000],
            'epsilon': [0.0001, 0.001, 0.01, 0.1, 1],
            'gamma': [0.0001, 0.001, 0.1, 1, 5]
        }
gsc = GridSearchCV(
        estimator=SVR(),
        param_grid=param_grid,
        cv=5, scoring=pear_score, 
        verbose=10000000, n_jobs=-1)

grid_result = gsc.fit(cv_train_dataframe, y_train)

best_params = grid_result.best_params_

print(best_params)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  

In [ ]:
from sklearn.svm import SVR
svr = SVR(**best_params)
svr.fit(cv_train_dataframe,y_train)

SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.001, gamma=0.001,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr,pearsonr

y_pred = svr.predict(cv_test_dataframe)
print(mean_squared_error(y_pred,y_test))

corr, _ = pearsonr(y_pred, y_test)
s_corr,_ = spearmanr(y_pred,y_test)
print("PEARSON: " + str(corr))

print("SPEARMAN: " + str(s_corr))

0.034659508291334816
PEARSON: 0.5321444052025485
SPEARMAN: 0.5359091299221972


#### Decision Trees

In [ ]:
param_grid = {"min_samples_split": [10, 20, 40],
              "max_depth": [2, 6, 8],
              "min_samples_leaf": [20, 40, 100],
              "max_leaf_nodes": [5, 20, 100],
              }

gsc = GridSearchCV(
        estimator=DecisionTreeRegressor(),
        param_grid=param_grid,
        cv=5, scoring=pear_score, 
        verbose=10000000, n_jobs=-1)

grid_result = gsc.fit(cv_train_dataframe, y_train)

best_params = grid_result.best_params_

print(best_params)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0925s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1904s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    0.8s
[Parallel(n_jo

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor(**best_params)
dtr.fit(cv_train_dataframe,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=8,
                      max_features=None, max_leaf_nodes=20,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=20, min_samples_split=20,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [ ]:
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr,pearsonr

y_pred = dtr.predict(cv_test_dataframe)
print(mean_squared_error(y_pred,y_test))

corr, _ = pearsonr(y_pred, y_test)
s_corr,_ = spearmanr(y_pred,y_test)
print("PEARSON: " + str(corr))

print("SPEARMAN: " + str(s_corr))

0.04290937072636934
PEARSON: 0.32373573840239056
SPEARMAN: 0.31600069112849316


Multi-Layer Perceptron

In [ ]:
from sklearn.neural_network import MLPRegressor
param_grid = {
    "hidden_layer_sizes": [(i,) for i in range(10,101,30)],
    "activation": [ "tanh", "relu"],
    "solver": ["adam"],
    "learning_rate":["invscaling", "adaptive"],
    "learning_rate_init":[0.005,0.0005,0.00005],
    "max_iter":[200,500,900],
}
print(param_grid)
gsc = GridSearchCV(
        estimator=MLPRegressor(),
        param_grid=param_grid,
        cv=5, scoring=pear_score, 
        verbose=10000000, n_jobs=-1)
grid_result = gsc.fit(cv_train_dataframe, y_train)


{'hidden_layer_sizes': [(10,), (40,), (70,), (100,)], 'activation': ['tanh', 'relu'], 'solver': ['adam'], 'learning_rate': ['invscaling', 'adaptive'], 'learning_rate_init': [0.005, 0.0005, 5e-05], 'max_iter': [200, 500, 900]}
Fitting 5 folds for each of 144 candidates, totalling 720 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.6s
[Parallel(n_job

In [ ]:
best_params = grid_result.best_params_

print(best_params)

{'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.0005, 'max_iter': 900, 'solver': 'adam'}


In [ ]:
mlp = MLPRegressor(**best_params)
mlp.fit(cv_train_dataframe,y_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='adaptive',
             learning_rate_init=0.0005, max_fun=15000, max_iter=900,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [ ]:
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr,pearsonr

y_pred = mlp.predict(cv_test_dataframe)
print(mean_squared_error(y_pred,y_test))

corr, _ = pearsonr(y_pred, y_test)
s_corr,_ = spearmanr(y_pred,y_test)
print("PEARSON: " + str(corr))

print("SPEARMAN: " + str(s_corr))

0.03528153057571035
PEARSON: 0.5133136769371818
SPEARMAN: 0.5056401976585773
